# Segment with geodesic active contour level set

In [ ]:
import sys
import os
from urllib.request import urlretrieve

import itk

from itkwidgets import view

In [ ]:
input_filename = 'BrainProtonDensitySlice.png'
if not os.path.exists(input_filename):
    url = 'https://data.kitware.com/api/v1/file/57b5d8028d777f10f2694bbf/download'
    urlretrieve(url, input_filename)

In [ ]:
InputPixelType = itk.ctype('float')

input_image = itk.imread(input_filename, InputPixelType)

view(input_image)

In [ ]:
smoothed = itk.curvature_anisotropic_diffusion_image_filter(input_image,
                                                            time_step=0.125,
                                                            number_of_iterations=5,
                                                            conductance_parameter=9.0)
view(smoothed)

In [ ]:
sigma = 1.0

gradient_magnitude = itk.gradient_magnitude_recursive_gaussian_image_filter(smoothed,
                                                                            sigma=sigma)
view(gradient_magnitude)

In [ ]:
alpha = -0.5
beta = 3.0

sigmoid = itk.sigmoid_image_filter(gradient_magnitude,
                                   output_minimum=0.0,
                                   output_maximum=1.0,
                                   alpha=alpha,
                                   beta=beta)

view(sigmoid)

In [ ]:
Dimension = input_image.GetImageDimension()
seeds = itk.VectorContainer[itk.UI, itk.LevelSetNode[InputPixelType, Dimension]].New()
seeds.Initialize()

seed_position = itk.Index[Dimension]()
seed_position[0] = 81
seed_position[1] = 114
node = itk.LevelSetNode[InputPixelType, Dimension]()
node.SetValue(-5.0)
node.SetIndex(seed_position)
seeds.InsertElement(0, node)

fast_marching = itk.fast_marching_image_filter(trial_points=seeds,
                                               speed_constant=1.0,
                                               output_size=input_image.GetBufferedRegion().GetSize())

In [ ]:
propagation_scaling = 2.0
number_of_iterations = 800

geodesic_active_contour = \
  itk.geodesic_active_contour_level_set_image_filter(fast_marching,
                                                     propagation_scaling=propagation_scaling,
                                                     curvature_scaling=1.0,
                                                     advection_scaling=1.0,
                                                     maximum_r_m_s_error=0.02,
                                                     number_of_iterations=number_of_iterations,
                                                     feature_image=sigmoid)

view(geodesic_active_contour)

In [ ]:
OutputPixelType = itk.ctype('unsigned char')
thresholded = itk.binary_threshold_image_filter(geodesic_active_contour,
                                                lower_threshold=-1000.0,
                                                upper_threshold=0.0,
                                                outside_value=itk.NumericTraits[OutputPixelType].min(),
                                                inside_value=itk.NumericTraits[OutputPixelType].max(),
                                                ttype=[type(geodesic_active_contour), itk.Image[OutputPixelType,Dimension]])

In [ ]:
view(thresholded)